<a href="https://colab.research.google.com/github/nosadchiy/public/blob/main/getExchangeRates_Frankfurter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import requests
from datetime import datetime, timedelta, date
import pandas as pd
from scipy.stats import multivariate_normal

In [11]:
import requests
from datetime import datetime, timedelta, date

def get_month_end_dates(start_date_str, end_date_str):
    """
    Generate a list of dates representing the last day of each month
    between start_date and end_date. If the end_date falls in the middle
    of a month, that date will be used for the final month.
    """
    start_date = datetime.strptime(start_date_str, '%Y-%m-%d').date()
    end_date   = datetime.strptime(end_date_str, '%Y-%m-%d').date()
    month_end_dates = []

    # Start from the first day of the month of start_date.
    current = start_date.replace(day=1)

    while current <= end_date:
        # Determine the first day of the next month.
        if current.month == 12:
            next_month = date(current.year + 1, 1, 1)
        else:
            next_month = date(current.year, current.month + 1, 1)

        # Last day of the current month is one day before the first day of the next month.
        last_day_of_month = next_month - timedelta(days=1)

        # If the last day of the month is after the specified end_date,
        # then use end_date (if it is in the same month) instead.
        if last_day_of_month > end_date:
            last_day_of_month = end_date

        if last_day_of_month >= start_date:
            month_end_dates.append(last_day_of_month)

        # Move to the first day of the next month.
        current = next_month

    return month_end_dates

def get_exchange_rate_for_date(date_str, base_currency="USD", symbols=None):
    """
    Fetches exchange rates for the given date from the Frankfurter API.

    :param date_str: The date string in 'YYYY-MM-DD' format.
    :param base_currency: The base currency (default "USD").
    :param symbols: A list of target currency codes.
    :return: A dictionary of exchange rates.
    """
    url = f"https://api.frankfurter.app/{date_str}"
    params = {"from": base_currency}
    if symbols:
        params["to"] = ",".join(symbols)

    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        return data.get("rates", {})
    else:
        print(f"Error fetching data for {date_str}: HTTP {response.status_code}")
        return {}

def estimate_statistics(exchange_data):
    """
    Given a dictionary of exchange rate data with dates as keys and
    rate dictionaries as values, this function computes:

      - The average exchange rate vector.
      - The variance-covariance matrix of the exchange rates.

    :param exchange_data: dict of the form {date_str: {currency: rate, ...}, ...}
    :return: A tuple (average_vector, covariance_matrix) where:
             - average_vector is a pandas Series.
             - covariance_matrix is a pandas DataFrame.
    """
    # Convert the nested dictionary to a pandas DataFrame.
    # Rows are indexed by date and columns are the currency codes.
    df = pd.DataFrame.from_dict(exchange_data, orient="index")
    df.index = pd.to_datetime(df.index)
    df.sort_index(inplace=True)

    # Calculate the average exchange rate for each currency.
    average_vector = df.mean()

    # Calculate the variance-covariance matrix of the exchange rates.
    covariance_matrix = df.cov()

    return average_vector, covariance_matrix



In [14]:
# Specify the time period and target currencies.
start_date_str = "2023-01-01"  # Change to your desired start date.
end_date_str   = "2024-12-31"  # Change to your desired end date.
target_currencies = ["BRL", "EUR", "INR", "JPY", "MXN"]

# Get a list of month-end dates for the given period.
month_end_dates = get_month_end_dates(start_date_str, end_date_str)

# Dictionary to hold the results.
exchange_data = {}

# Fetch the exchange rates for each month-end date.
for d in month_end_dates:
    date_str = d.isoformat()
    print(f"Fetching rates for {date_str}...")
    rates = get_exchange_rate_for_date(date_str, base_currency="USD", symbols=target_currencies)
    exchange_data[date_str] = rates

# Display the collected exchange rate data.
print("\nExchange Rates at Month End:")
for date_str, rates in exchange_data.items():
    print(f"{date_str}: {rates}")

# Estimate average exchange rate vector and the variance-covariance matrix.
avg_vector, cov_matrix = estimate_statistics(exchange_data)

print("\nAverage Exchange Rate Vector:")
print(avg_vector)

print("\nVariance-Covariance Matrix:")
print(cov_matrix)






Fetching rates for 2023-01-31...
Fetching rates for 2023-02-28...
Fetching rates for 2023-03-31...
Fetching rates for 2023-04-30...
Fetching rates for 2023-05-31...
Fetching rates for 2023-06-30...
Fetching rates for 2023-07-31...
Fetching rates for 2023-08-31...
Fetching rates for 2023-09-30...
Fetching rates for 2023-10-31...
Fetching rates for 2023-11-30...
Fetching rates for 2023-12-31...
Fetching rates for 2024-01-31...
Fetching rates for 2024-02-29...
Fetching rates for 2024-03-31...
Fetching rates for 2024-04-30...
Fetching rates for 2024-05-31...
Fetching rates for 2024-06-30...
Fetching rates for 2024-07-31...
Fetching rates for 2024-08-31...
Fetching rates for 2024-09-30...
Fetching rates for 2024-10-31...
Fetching rates for 2024-11-30...
Fetching rates for 2024-12-31...

Exchange Rates at Month End:
2023-01-31: {'BRL': 5.1115, 'EUR': 0.92311, 'INR': 81.82, 'JPY': 130.41, 'MXN': 18.8277}
2023-02-28: {'BRL': 5.2058, 'EUR': 0.94171, 'INR': 82.61, 'JPY': 136.76, 'MXN': 18.3143}


In [16]:
mvn_distribution = multivariate_normal(mean=avg_vector, cov=cov_matrix)
sample = mvn_distribution.rvs(size=1, random_state = 1)

print("Sampled draw:", sample)

Sampled draw: [  5.06667254   0.8999488   82.71303795 133.8740746   18.64681926]
